In [1]:
import pandas as pd
import numpy as np
import MySQLdb
import datetime

In [20]:
company_order = pd.read_csv('myTempCompanyAll20170915_004.csv',encoding='gbk',header=None,index_col=None)

In [16]:
db = MySQLdb.connect("47.100.14.137","hyps44413","Nanhua")
db.set_character_set('gbk')

In [79]:
db = MySQLdb.connect("localhost","root","123456")
db.set_character_set('gbk')

In [160]:
V_info_create()
#B_info_create()
#S_info_create()

In [112]:
create_futures_member()
create_non_futures_member()
create_total_volume()

NameError: name 'creat_futures_member' is not defined

In [77]:
db.close()

In [80]:
now_date = datetime.datetime(2017,10,30)
end_date = datetime.datetime(2017,11,22)

databaseName = 'i_oi_data'

while now_date <= end_date:
    split_date = now_date.strftime('%Y-%m-%d').split('-')
    table_date = split_date[0]+split_date[1]+split_date[2]
    try:
        data = pd.read_csv('../大商所/铁矿石/'+table_date+'.csv',encoding='gbk',engine='python')
    except Exception as e:
        now_date = now_date + datetime.timedelta(days = 1)
        print (e)
        continue
        
    for i in range(3,len(data)-1):
        for j in range(12):
            if(data.loc[i][j] == "-"):
                data.loc[i][j] = 'NULL'                
            elif( (j==2 or j==3 or j==6 or j==7 or j==10 or j==11) and data.loc[i][j] != "NULL"):
                data.loc[i][j] = data.loc[i][j].replace(',','')
  
    tmp = now_date

    #total_insert(data,table_date,databaseName)
    #futures_member_insert(data,table_date,databaseName)
    #non_futures_member_insert(data,table_date,databaseName)
    #total_volume_insert(data,table_date,databaseName)
        
    V_info_insert(data,table_date,databaseName)
    #B_info_insert(data,table_date,databaseName)
    #S_info_insert(data,table_date,databaseName)
    
    now_date = now_date + datetime.timedelta(days = 1)
    
db.close()

[Errno 2] No such file or directory: '../大商所/铁矿石/20171104.csv'
[Errno 2] No such file or directory: '../大商所/铁矿石/20171105.csv'
[Errno 2] No such file or directory: '../大商所/铁矿石/20171111.csv'
[Errno 2] No such file or directory: '../大商所/铁矿石/20171112.csv'
[Errno 2] No such file or directory: '../大商所/铁矿石/20171118.csv'
[Errno 2] No such file or directory: '../大商所/铁矿石/20171119.csv'


In [6]:
def total_insert(data,table_date,databaseName):       
    cursor = db.cursor()
    
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".i_" + table_date+ "(\
    Rank INT(5),\
    Member CHAR(10),\
    Volume FLOAT(5),\
    Increment FLOAT(5),\
    Rank_1 INT(5),\
    Member1 CHAR(10),\
    Buy_Volume FLOAT(5),\
    Increment_1 FLOAT(5),\
    Rank_2 INT(5),\
    Member2 CHAR(10),\
    Sales_Volume FLOAT(5),\
    Increment_2 FLOAT(5) )"

    cursor.execute(sql)
    value=[]

    for i in range(3,len(data)-1):      
        value.append((data.loc[i][0],data.loc[i][1],data.loc[i][2],data.loc[i][3],data.loc[i][4],data.loc[i][5],data.loc[i][6],data.loc[i][7],data.loc[i][8],data.loc[i][9],data.loc[i][10],data.loc[i][11]))

    sql = "INSERT INTO " + databaseName + ".i_" + table_date +"(Rank,Member,Volume,Increment,Rank_1,Member1,Buy_Volume,Increment_1 \
    ,Rank_2,Member2,Sales_Volume,Increment_2) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        
    cursor.executemany(sql,value)
    db.commit()
    cursor.close()

# 期貨公司成員輸入

In [113]:
def create_futures_member():
    cursor = db.cursor()
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".futures_member (\
    Date DATE PRIMARY KEY,\
    Total_volume FLOAT(5),\
    Increment FLOAT(5),\
    Total_buy_volume FLOAT(5),\
    Increment_1 FLOAT(5),\
    Total_sales_volume FLOAT(5),\
    Increment_2 FLOAT(5) ) "

    cursor.execute(sql)

In [5]:
def futures_member_insert(data,table_date,databaseName):
    
    sql = "INSERT INTO " + databaseName + ".futures_member (Date,Total_volume,Increment,Total_buy_volume,Increment_1,Total_sales_volume,Increment_2) VALUES( "\
    + table_date +","\
    + data.loc[0][1].replace(',','') +","\
    + data.loc[0][2].replace(',','') +","\
    + data.loc[0][3].replace(',','') +","\
    + data.loc[0][4].replace(',','') +","\
    + data.loc[0][5].replace(',','') +","\
    + data.loc[0][6].replace(',','') + ")"

    cursor.execute(sql)
    db.commit()
    cursor.close()

# 非期貨公司成員輸入

In [114]:
def create_non_futures_member():
    cursor = db.cursor()
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".none_futures_member (\
    Date DATE PRIMARY KEY,\
    Total_volume FLOAT(5),\
    Increment FLOAT(5),\
    Total_buy_volume FLOAT(5),\
    Increment_1 FLOAT(5),\
    Total_sales_volume FLOAT(5),\
    Increment_2 FLOAT(5) ) "

    cursor.execute(sql)

In [21]:
def non_futures_member_insert(data,table_date,databaseName):
    cursor = db.cursor()
    sql = "INSERT INTO " + databaseName +".none_futures_member (Date,Total_volume,Increment,Total_buy_volume,Increment_1,Total_sales_volume,Increment_2) VALUES("\
    + table_date + ","\
    + data.loc[1][1].replace(',','') +","\
    + data.loc[1][2].replace(',','') +","\
    + data.loc[1][3].replace(',','') +","\
    + data.loc[1][4].replace(',','') +","\
    + data.loc[1][5].replace(',','') +","\
    + data.loc[1][6].replace(',','') + ")"

    cursor.execute(sql)
    db.commit()
    cursor.close()

# 總計

In [115]:
def create_total_volume():
    cursor = db.cursor()
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".total_volume ( Date DATE PRIMARY KEY, \
    V_Valume FLOAT(5),V_Increment FLOAT(5),\
    B_Valume FLOAT(5),B_Increment FLOAT(5),\
    S_Valume FLOAT(5),S_Increment FLOAT(5))"

    cursor.execute(sql)

In [25]:
def total_volume_insert(data,table_date,databaseName):
    cursor = db.cursor()
    total_len = len(data)-1
    sql = "INSERT INTO " + databaseName + ".total_volume (Date,V_Valume,V_Increment,B_Valume,B_Increment,S_Valume,S_Increment) VALUES("\
    + table_date + ","\
    + data.loc[total_len][2].replace(',','') +","\
    + data.loc[total_len][3].replace(',','') +","\
    + data.loc[total_len][6].replace(',','') +","\
    + data.loc[total_len][7].replace(',','') +","\
    + data.loc[total_len][10].replace(',','') +","\
    + data.loc[total_len][11].replace(',','') + ")"

    cursor.execute(sql)
    db.commit()

# 各公司資訊

## V名次、成交量、增減

In [45]:
def V_info_create():
    cursor = db.cursor()
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' CHAR(10),'
    name = name + company_order.loc[len(company_order)-1][0] + " CHAR(10)"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".v_rank ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".v_volume ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".v_increment ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)
    cursor.close()

In [71]:
data

,会员类别,总成交量,增减,总持买单量,增减.1,总持卖单量,增减.2,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,期货公司会员,"2,381,042","-1,173,118","1,037,000","-75,569","1,037,000","-75,569",NaN,NaN,NaN,NaN,NaN
1,非期货公司会员,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2,名次,会员简称,成交量,增减,名次,会员简称,持买单量,增减,名次,会员简称,持卖单量,增减
3,1,国投安信,229785,-43168,1,银河期货,73251,-1879,1,中信期货,107529,-5641
4,2,海通期货,172213,-45990,2,中信期货,72645,-2910,2,永安期货,81404,16015
5,3,银河期货,97388,-55844,3,海通期货,56734,-9509,3,国投安信,48497,-3709
6,4,东证期货,88462,-34276,4,永安期货,55531,3056,4,海通期货,47396,-10756
7,5,招商期货,87158,-13800,5,国投安信,52143,-3308,5,方正中期,45632,-64
8,6,申银万国,82004,-43512,6,国泰君安,42545,-267,6,银河期货,45474,-5847
9,7,中信期货,76535,-23702,7,华泰期货,31517,475,7,国泰君安,32486,928


In [72]:
def V_info_insert(data,table_date,databaseName):
    cursor = db.cursor()
    
    empty = ""
    empty = empty + table_date


    sql = "INSERT INTO " + databaseName + ".v_rank (Date) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".v_volume (Date) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".v_increment (Date) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(3,len(data)-1):
        if(data.loc[i][1] == 'NULL'):
            continue
        
        else:
            sql = "UPDATE " + databaseName + ".v_rank SET " + str(data.loc[i][1]) + "=" + data.loc[i][0] + " WHERE Date=" + table_date
            cursor.execute(sql)
            db.commit()    

    for i in range(3,len(data)-1):
        if(data.loc[i][1] == 'NULL'):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".v_volume SET " + str(data.loc[i][1]) + "=" + data.loc[i][2] + " WHERE Date=" + table_date
            cursor.execute(sql)
            db.commit()

    for i in range(3,len(data)-1):
        if(data.loc[i][1] == 'NULL'):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".v_increment SET " + data.loc[i][1] + "=" + data.loc[i][3] + " WHERE Date=" + table_date
            cursor.execute(sql)
            db.commit()
    
    cursor.close()

## B名次、買單量、增減

In [198]:
def B_info_create():
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' CHAR(10),'
    name = name + company_order.loc[len(company_order)-1][0] + " CHAR(10)"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_rank ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_oi ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_increment ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)

0

In [199]:
def B_info_insert(data,table_date,list_date,databaseName)
    name=""

    empty = ""
    empty = empty + table_date 
    
    sql = "INSERT INTO " + databaseName + ".b_rank (Date," + name + ") VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".b_oi (Date," + name + ") VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".b_increment (Date," + name + ") VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(3,len(data)-1):
    if(data.loc[i][1] == "NULL"):
        continue  
        
    sql = "UPDATE " + databaseName + ".b_rank SET " + str(data.loc[i][5]) + "=" + data.loc[i][4] + " WHERE Date='" + list_date + "'"
    cursor.execute(sql)
    db.commit()    

    for i in range(3,len(data)-1):
        if(data.loc[i][5] == "NULL"):
            continue
        sql = "UPDATE " + databaseName + ".b_oi SET " + str(data.loc[i][5]) + "=" + data.loc[i][6] + " WHERE Date='" + list_date + "'"
        cursor.execute(sql)
        db.commit()

    for i in range(3,len(data)-1):
        if(data.loc[i][9] == "NULL"):
            continue
        sql = "UPDATE " + databaseName + ".b_increment SET " + data.loc[i][5] + "=" + data.loc[i][7] + " WHERE Date='" + list_date + "'"
        cursor.execute(sql)
        db.commit()

    cursor.close()

## S名次、賣單量、增減

In [203]:
def S_info_create():
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' CHAR(10),'
    name = name + company_order.loc[len(company_order)-1][0] + " CHAR(10)"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".s_rank ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".s_oi ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".s_increment ( Date DATE PRIMARY KEY, " + name + ")"
    cursor.execute(sql)

0

In [204]:
def S_info_insert(data,table_date,list_date,databaseName):
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ','
    name = name + company_order.loc[len(company_order)-1][0]

    empty = ""
    empty = empty + table_date + ","
    for i in range(len(company_order)-1):
        empty = empty + "NULL" + ','
    empty = empty + "NULL"
    
    sql = "INSERT INTO " + databaseName + ".s_rank (Date," + name + ") VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".s_oi (Date," + name + ") VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".s_increment (Date," + name + ") VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(3,len(data)-1):
    if(data.loc[i][1] == "NULL"):
        continue  
        
    sql = "UPDATE " + databaseName + ".s_rank SET " + str(data.loc[i][1]) + "=" + data.loc[i][0] + " WHERE Date='" + list_date + "'"
    cursor.execute(sql)
    db.commit()    

    for i in range(3,len(data)-1):
        if(data.loc[i][5] == "NULL"):
            continue
        sql = "UPDATE " + databaseName + ".s_oi SET " + str(data.loc[i][1]) + "=" + data.loc[i][2] + " WHERE Date='" + list_date + "'"
        cursor.execute(sql)
        db.commit()

    for i in range(3,len(data)-1):
        if(data.loc[i][9] == "NULL"):
            continue
        sql = "UPDATE " + databaseName +".s_increment SET " + data.loc[i][9] + "=" + data.loc[i][3] + " WHERE Date='" + list_date + "'"
        cursor.execute(sql)
        db.commit()

    cursor.close()